# HiQuER Toolchain Example

### Optimize Circuit

We run a staq optimization pass to reduce the circuit size. First, some stats:

In [4]:
%%bash

cat qasm/plasma.qasm | staq_resource_estimator

Resources used:
  rz(-0.0181802): 2
  rz(-0.0128553): 4
  rz(-0.0203261): 2
  qubits: 8
  h: 168
  rz(-0.0140267): 2
  rz(0.00724338): 4
  rz(0.0114528): 2
  s: 168
  rz(0.00991841): 4
  depth: 196
  rz(0.0151506): 2
  rz(0.0222661): 2
  rz(0.0128046): 2
  rz(0.00809835): 4
  cx: 168
  rz(-0.0157445): 4
  rz(-0.0114982): 4
  rz(-0.0240501): 2


In [6]:
%%bash

staq -O2 qasm/plasma.qasm | tee qasm/plasma_opt.qasm | staq_resource_estimator

Resources used:
  depth: 150
  rz(-0.0181802): 2
  rz(0.0222661): 2
  rz(-0.0257106): 2
  qubits: 8
  cx: 144
  rz(-0.0140267): 2
  rz(0.0114528): 2
  s: 110
  rz(0.0144868): 2
  rz(0.0198368): 2
  rz(-0.00889949): 2
  rz(-0.0229963): 2
  h: 118
  rz(0.0161967): 2
  rz(-0.031489): 2
  rz(-0.00752144): 2


### Convert to Clifford+T

In [10]:
%%bash

cat qasm/plasma_opt.qasm | python3 ../util/rotation_synth/rotation_synth.py | \
tee qasm/plasma_CT.qasm | staq_resource_estimator

Resources used:
  depth: 3308
  x: 1
  y: 4
  s: 954
  t: 1686
  cx: 144
  h: 1800
  z: 5
  qubits: 8


## Lattice Surgery

In [13]:
%%bash

cat qasm/plasma_CT.qasm | staq_lattice_surgery > qasm/plasma_ls.json

## Resource Estimation

In [18]:
%%bash

cat qasm/plasma_ls.json | python3 ../util/qre/src/staq_qre.py -c ../util/qre/src/config_fast.json

{
    "logical": {
        "n_qubits": 8,
        "T-count": 1686,
        "Commuting T-layers": 973
    },
    "qecc": {
        "scheme": "fast",
        "p_g": 0.001,
        "cycle_time": 1e-06
    },
    "physical": {
        "physical qubits": 29150,
        "time (sec)": 0.018546,
        "time (min)": 0.0003091,
        "time (hours)": 5.151666666666667e-06,
        "time (days)": 2.1465277777777778e-07,
        "time (months)": 7.155092592592593e-09,
        "time (years)": 5.962577160493828e-10,
        "distance used": 11,
        "fraction factories": 0.79,
        "number of factories": 5,
        "factory used": {
            "p_phys": 0.001,
            "p_out": 4.5e-08,
            "qubits": 4620,
            "cycles": 42.6
        }
    }
}
